# Run Bee World Environment

In [1]:
import gymnasium as gym
from bee import BeeWorld

import numpy as np
import sonnet as snt
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from IPython.display import HTML


In [2]:
def display_video(frames, framerate=30):
  """Generates video from `frames`.

  Args:
    frames (ndarray): Array of shape (n_frames, height, width, 3).
    framerate (int): Frame rate in units of Hz.

  Returns:
    Display object.
  """
  height, width, _ = frames[0].shape
  dpi = 70
  orig_backend = matplotlib.get_backend()
  matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
  fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
  matplotlib.use(orig_backend)  # Switch back to the original backend.
  ax.set_axis_off()
  ax.set_aspect('equal')
  ax.set_position([0, 0, 1, 1])
  im = ax.imshow(frames[0])
  
  def update(frame):
    im.set_data(frame)
    return [im]
  interval = 1000/framerate
  anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                  interval=interval, blit=True, repeat=False)
  return HTML(anim.to_html5_video())

In [3]:
gym.register(
    id="BeeWorld",
    entry_point=BeeWorld,
    max_episode_steps=3000,
)

env = gym.make("BeeWorld", render_mode="rgb_array")
env.reset()

({'vision': 0, 'smell': array([0.67234631])}, {})

In [4]:
# Run env for n_steps, apply random actions, and show video.
n_steps = 200

frames = []
timestep = env.reset()
for _ in range(n_steps):
    A = np.random.uniform(-0.5, 0.5)
    theta = np.random.uniform(-0.5, 0.5)
    env.step((A, theta))
    frames.append(env.render())

env.close()

  

In [5]:
display_video(frames)